### Aluna: Cristiane Brasil Ueda 

## Apresentar a formulação 

$$
f_{\text{25}}(\mathbf{x}) \;=\; \sum_{i=1}^{D-1} \left[ 50\left(x_{i+1} - x_i^2\right)^2 \;+\; \left(x_i - 2\right)^2 \right]
$$


In [1]:
import numpy as np
import time
from scipy.optimize import minimize

# Função modificada com coeficiente 50 e (x_i - 2)^2
def custom_rosenbrock(x):
    fval = sum(50.0 * (x[1:] - x[:-1]**2.0)**2.0 + (x[:-1] - 2)**2.0)

    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    grad = np.zeros_like(x)
    grad[1:-1] = 100 * (xm - xm_m1**2) - 200 * (xm_p1 - xm**2) * xm + 2 * (xm - 2)
    grad[0] = -200 * x[0] * (x[1] - x[0]**2) + 2 * (x[0] - 2)
    grad[-1] = 100 * (x[-1] - x[-2]**2)

    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        if i > 0:
            H[i, i - 1] = -200 * x[i - 1]
        if i < n - 1:
            H[i, i] = 600 * x[i]**2 - 200 * x[i + 1] + 2
            H[i, i + 1] = -200 * x[i]
        else:
            H[i, i] = 100
    for i in range(n - 1):
        H[i + 1, i] = H[i, i + 1]

    return fval, grad, H

# Ponto inicial
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

# Métodos a serem comparados
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: custom_rosenbrock(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: custom_rosenbrock(x)[1], "hess": lambda x: custom_rosenbrock(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: custom_rosenbrock(x)[1], "hess": lambda x: custom_rosenbrock(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: custom_rosenbrock(x)[1], "hess": lambda x: custom_rosenbrock(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: custom_rosenbrock(x)[1], "hess": lambda x: custom_rosenbrock(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: custom_rosenbrock(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Custom Rosenbrock Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")



==== Custom Rosenbrock Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   1.6971e+00 |    623 |    N/A |    N/A |      26.68 |     True
BFGS                 |   1.6971e+00 |     40 |     40 |    N/A |       8.76 |     True
Newton-CG            |   2.2584e+00 |   1003 |   1003 |   1000 |     138.86 |    False
trust-ncg            |   1.6972e+00 |   1001 |    966 |    965 |     217.15 |    False
trust-krylov         |   1.6971e+00 |    310 |    310 |    307 |     308.68 |     True
trust-exact          |   1.6971e+00 |    135 |    133 |    135 |     104.80 |     True


$$
\min_{x_1, x_2} \Bigl(\cos x_1 ,\sin x_2 ;-; \frac{x_1}{x_2^2 + 1}\Bigr) $$

In [8]:
def ex02(x):
    x1, x2 = x

    # Função
    f = np.cos(x1) * np.sin(x2) - x1 / (x2**2 + 1)

    # Gradiente
    df_dx1 = -np.sin(x1) * np.sin(x2) - 1 / (x2**2 + 1)
    df_dx2 = np.cos(x1) * np.cos(x2) + (2 * x1 * x2) / (x2**2 + 1)**2
    g = np.array([df_dx1, df_dx2])

    # Hessiana
    d2f_dx1dx1 = -np.cos(x1) * np.sin(x2)
    d2f_dx1dx2 = -np.sin(x1) * np.cos(x2) + (2 * x2) / (x2**2 + 1)**2
    d2f_dx2dx1 = d2f_dx1dx2
    d2f_dx2dx2 = -np.cos(x1) * np.sin(x2) + \
                 (2 * x1 * (x2**2 + 1)**2 - 8 * x1 * x2**2 * (x2**2 + 1)) / (x2**2 + 1)**4
    H = np.array([
        [d2f_dx1dx1, d2f_dx1dx2],
        [d2f_dx2dx1, d2f_dx2dx2]
    ])

    return f, g, H

# Initial point
x0 = np.array([0.0, 0.0])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ex02(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ex02(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 02 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 02 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |  -1.0457e+00 |    167 |    N/A |    N/A |       7.28 |     True
BFGS                 |  -9.6673e+22 |    248 |    236 |    N/A |      22.83 |    False
Newton-CG            |   0.0000e+00 |      1 |      1 |      1 |       0.00 |     True
trust-ncg            |  -1.6960e+05 |    399 |    396 |    395 |      36.76 |    False
trust-krylov         |   0.0000e+00 |      2 |      2 |      1 |       2.00 |    False
trust-exact          |  -2.2003e+05 |    401 |    389 |    401 |      97.67 |    False


Problem Statistics: 

\# of continuous variables: 2

\# of known solutions: 3

Global solution:

Objective function: -2.02181

Continuous variables: $x_1 = 2; x_2 = 0.10578$

In [9]:
results['trust-exact']

{'x0': array([0., 0.]),
 'x*': array([2.20032125e+05, 6.37081271e-04]),
 'fval': np.float64(-220032.03547900406),
 'nfev': 401,
 'njev': 389,
 'nhev': 401,
 'time': 97.67484664916992,
 'success': False}

In [10]:
ex02(np.array([2., 0.10578]))

(np.float64(-2.0218067833370204),
 array([-1.08494061e+00, -1.31240428e-05]),
 array([[ 0.04393797, -0.69731109],
        [-0.69731109,  3.78275021]]))

Testes de derivadas por diferenças finitas

In [11]:
# Gradiente e Hessiana numéricas
from scipy.optimize import approx_fprime
def numerical_hessian(f_grad, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    fx = f_grad(x)
    for i in range(n):
        x1 = x.copy()
        x1[i] += h
        f1 = f_grad(x1)
        x2 = x.copy()
        x2[i] -= h
        f2 = f_grad(x2)
        H[:, i] = (f1 - f2) / (2 * h)
    return H

# Ponto de teste
x0 = np.array([1.0, 1.0])
eps = np.sqrt(np.finfo(float).eps)

# Avaliação da função
f_val, g_analytical, H_analytical = ex02(x0)

# Gradiente numérico
g_numeric = approx_fprime(x0, lambda x: ex02(x)[0], eps)

# Hessiana numérica via gradiente
H_numeric = numerical_hessian(lambda x: ex02(x)[1], x0)

# Diferenças absolutas
grad_diff = np.abs(g_numeric - g_analytical)
hess_diff = np.abs(H_numeric - H_analytical)

grad_diff, hess_diff


(array([7.76445575e-09, 1.14897460e-08]),
 array([[2.34168240e-12, 5.61722197e-12],
        [5.61722197e-12, 5.45773426e-11]]))

$$
\min_{x, y} 
\Bigl[
1 
+ (x + y + 1)^2 \,\bigl(19 - 14x + 3x^2 - 14y + 6xy + 3y^2\bigr)
\Bigr]
\;\times\;
\Bigl[
30 
+ (2x - 3y)^2 \,\bigl(18 - 32x^2 + 12x^2 + 48y - 36xy + 27y^2\bigr)
\Bigr]
$$

In [12]:
import numpy as np

# Função e gradiente analítico
def goldstein_price_fg(x):
    x1, x2 = x
    a = x1 + x2 + 1
    b = 2 * x1 - 3 * x2

    A = 1 + a**2 * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2)
    B = 30 + b**2 * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2)

    f = A * B

    dA_dx1 = 2 * a * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2) \
             + a**2 * (-14 + 6 * x1 + 6 * x2)
    dA_dx2 = 2 * a * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2) \
             + a**2 * (-14 + 6 * x1 + 6 * x2)

    dB_dx1 = 4 * b * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2) \
             + b**2 * (-32 + 24 * x1 - 36 * x2)
    dB_dx2 = -6 * b * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2) \
             + b**2 * (48 - 36 * x1 + 54 * x2)

    df_dx1 = dA_dx1 * B + A * dB_dx1
    df_dx2 = dA_dx2 * B + A * dB_dx2
    g = np.array([df_dx1, df_dx2])

    return f, g

# Hessiana numérica com diferenças centrais
def numerical_hessian(grad_func, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += h
        x2[i] -= h
        g1 = grad_func(x1)
        g2 = grad_func(x2)
        H[:, i] = (g1 - g2) / (2 * h)
    return H

# Interface principal
def ex03(x):
    f, g = goldstein_price_fg(x)
    H = numerical_hessian(lambda x_: goldstein_price_fg(x_)[1], x)
    return f, g, H



In [13]:
x0 = np.array([0.0, -1.0])
fval, grad, hess = ex03(x0)
print("f(x) =", fval)
print("∇f(x) =", grad)
print("∇²f(x) =\n", hess)

f(x) = 3.0
∇f(x) = [0. 0.]
∇²f(x) =
 [[ 504.0000021  -215.99999929]
 [-216.00000055  864.00000477]]


In [14]:
# Initial point
x0 = np.array([0.0, 0.0])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ex03(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ex03(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 03 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 03 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.0000e+01 |    121 |    N/A |    N/A |       9.51 |     True
BFGS                 |   3.0000e+01 |     16 |     16 |    N/A |       3.62 |     True
Newton-CG            |   3.0000e+01 |     13 |     13 |      8 |       3.00 |     True
trust-ncg            |   3.0000e+01 |     11 |      9 |      8 |       2.28 |     True
trust-krylov         |   3.0000e+01 |      7 |      7 |      6 |       5.40 |     True
trust-exact          |   3.0000e+01 |      7 |      7 |      7 |       1.47 |     True


Problem Statistics: 

\# of continuous variables: 2

\# of known solutions: 4

Global solution:

Objective function: 3

Continuous variables: $x_1 = 0.0; x_2 = -1.0$

$$
f_1(x) \;=\; -20 \, e^{-0.2 \,\sqrt{\frac{1}{D}\,\sum_{i=1}^{D} x_i^2}} 
\;-\; e^{\frac{1}{D}\,\sum_{i=1}^{D} \cos\bigl(2\pi x_i\bigr)} 
\;+\; 20 
\;+\; e
$$

In [15]:

# Valor e gradiente da função Ackley 1
def ackley1_fg(x):
    D = len(x)
    sum_sq = np.sum(x**2)
    sum_cos = np.sum(np.cos(2 * np.pi * x))
    
    term1 = -20 * np.exp(-0.2 * np.sqrt(sum_sq / D))
    term2 = -np.exp(sum_cos / D)
    f = term1 + term2 + 20 + np.e

    # Gradiente analítico
    sqrt_sum_sq = np.sqrt(sum_sq / D)
    if sqrt_sum_sq == 0:
        grad1 = 0
    else:
        grad1 = (4 * x / (D * sqrt_sum_sq)) * np.exp(-0.2 * sqrt_sum_sq)

    grad2 = (2 * np.pi / D) * np.sin(2 * np.pi * x) * np.exp(sum_cos / D)

    g = grad1 + grad2
    return f, g

# Hessiana numérica usando gradiente externo
def numerical_hessian(grad_func, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += h
        x2[i] -= h
        g1 = grad_func(x1)
        g2 = grad_func(x2)
        H[:, i] = (g1 - g2) / (2 * h)
    return H

# Interface final
def ackley1(x):
    f, g = ackley1_fg(x)
    H = numerical_hessian(lambda x_: ackley1_fg(x_)[1], x)
    return f, g, H



In [10]:
x0 = np.zeros(5)
fval, grad, hess = ackley1(x0)
print("f(x) =", fval)
print("∇f(x) =", grad)
print("∇²f(x) =\n", hess)

f(x) = 4.440892098500626e-16
∇f(x) = [0. 0. 0. 0. 0.]
∇²f(x) =
 [[178906.74089307      0.              0.              0.
       0.        ]
 [     0.         178906.74089307      0.              0.
       0.        ]
 [     0.              0.         178906.74089307      0.
       0.        ]
 [     0.              0.              0.         178906.74089307
       0.        ]
 [     0.              0.              0.              0.
  178906.74089307]]


In [16]:
# Initial point
x0 = np.ones(5)

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ackley1(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ackley1(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 03 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 03 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.5745e+00 |    145 |    N/A |    N/A |      35.04 |     True
BFGS                 |   3.5745e+00 |      7 |      7 |    N/A |       4.90 |     True
Newton-CG            |   3.5745e+00 |      4 |      4 |      3 |       2.25 |     True
trust-ncg            |   3.5745e+00 |      3 |      3 |      2 |       1.72 |     True
trust-krylov         |   3.5745e+00 |      3 |      3 |      2 |       4.21 |     True
trust-exact          |   3.5745e+00 |      3 |      3 |      3 |       2.34 |     True


## Exercício 1 - Função 105 - Rosenbrock

Avaliar para diferentes valores de D

## Apresentar a formulação 

$$
f_{\text{25}}(\mathbf{x}) \;=\; \sum_{i=1}^{D-1} \left[ 50\left(x_{i+1} - x_i^2\right)^2 \;+\; \left(x_i - 2\right)^2 \right]
$$


In [2]:
import numpy as np
import time
from scipy.optimize import minimize

# Função f_25 com gradiente e hessiano
def f25(x):
    fval = sum(50.0 * (x[1:] - x[:-1]**2.0)**2.0 + (x[:-1] - 2)**2.0)

    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    grad = np.zeros_like(x)
    grad[1:-1] = 100 * (xm - xm_m1**2) - 200 * (xm_p1 - xm**2) * xm + 2 * (xm - 2)
    grad[0] = -200 * x[0] * (x[1] - x[0]**2) + 2 * (x[0] - 2)
    grad[-1] = 100 * (x[-1] - x[-2]**2)

    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        if i > 0:
            H[i, i - 1] = -200 * x[i - 1]
        if i < n - 1:
            H[i, i] = 600 * x[i]**2 - 200 * x[i + 1] + 2
            H[i, i + 1] = -200 * x[i]
        else:
            H[i, i] = 100
    for i in range(n - 1):
        H[i + 1, i] = H[i, i + 1]

    return fval, grad, H

# Métodos de otimização
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: f25(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: f25(x)[1], "hess": lambda x: f25(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: f25(x)[1], "hess": lambda x: f25(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: f25(x)[1], "hess": lambda x: f25(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: f25(x)[1], "hess": lambda x: f25(x)[2]},
}

# Dicionário para armazenar os tempos de cada método
method_times = {method: [] for method in methods}

# Executa as otimizações para D = 6 a 10
for D in range(6, 11):
    print(f"\n==== f_25 Results for D = {D} ====\n")
    x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
    if len(x0) < D:
        x0 = np.concatenate([x0, np.ones(D - len(x0))])
    else:
        x0 = x0[:D]

    results = {}
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: f25(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000  # em ms
        method_times[name].append(elapsed_time)

        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }

    # Apresenta os resultados
    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")

# Cálculo da média dos tempos
print("\n==== Média de Tempo por Método (D = 6 a 10) ====\n")
print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
print("-" * 38)
for method, times in method_times.items():
    avg_time = sum(times) / len(times)
    print(f"{method:<20} | {avg_time:15.2f}")



==== f_25 Results for D = 6 ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   2.5821e+00 |    873 |    N/A |    N/A |      34.92 |     True
BFGS                 |   2.5821e+00 |     53 |     53 |    N/A |       5.51 |     True
Newton-CG            |   3.1344e+00 |   1204 |   1204 |   1200 |     175.35 |    False
trust-ncg            |   2.5843e+00 |   1201 |   1183 |   1182 |     191.92 |    False
trust-krylov         |   2.5821e+00 |    317 |    317 |    314 |     222.15 |     True
trust-exact          |   2.5821e+00 |    132 |    130 |    132 |     110.83 |     True

==== f_25 Results for D = 7 ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.5161e+00 |   1180 |    N/A |    N

## Exercício 2- Função 114 - Scahffer 3

Testar para diferentes pontos iniciais.

$$
f_3(x_1, x_2) = 0.5 + 
\frac{\cos^2\left(x_1 \cdot \sin(x_2)\right) - 0.5}
{\left[1 + 0.001\left(x_1^2 + x_2^2\right)\right]^2}
$$


In [5]:
import numpy as np
import time
from scipy.optimize import minimize

# Função Schaffer N.3 modificada
def schaffer_cos_variant(x):
    x1, x2 = x[0], x[1]

    # Função
    u = x1 * np.sin(x2)
    numerator = np.cos(u)**2 - 0.5
    denominator = (1 + 0.001 * (x1**2 + x2**2))**2
    f = 0.5 + numerator / denominator

    # Gradiente
    du_dx1 = np.sin(x2)
    du_dx2 = x1 * np.cos(x2)

    dnumer_dx1 = -2 * np.cos(u) * np.sin(u) * du_dx1
    dnumer_dx2 = -2 * np.cos(u) * np.sin(u) * du_dx2

    ddenom_dx1 = 2 * (1 + 0.001 * (x1**2 + x2**2)) * 0.002 * x1
    ddenom_dx2 = 2 * (1 + 0.001 * (x1**2 + x2**2)) * 0.002 * x2

    dfdx1 = (dnumer_dx1 * denominator - numerator * ddenom_dx1) / denominator**2
    dfdx2 = (dnumer_dx2 * denominator - numerator * ddenom_dx2) / denominator**2

    g = np.array([dfdx1, dfdx2])

    # Hessiana (simplificada — opcional melhorar conforme a precisão desejada)
    H = np.zeros((2, 2))  # Pode-se adicionar o cálculo exato se necessário

    return f, g, H

# Lista de pontos iniciais
initial_points = [
    np.array([2.5, 1.75]),
    np.array([0.0, 0.0]),
    np.array([3.0, -4.0]),
    np.array([-1.5, 2.5]),
    np.array([5.0, 5.0])
]

# Métodos de otimização
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: schaffer_cos_variant(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: schaffer_cos_variant(x)[1], "hess": lambda x: schaffer_cos_variant(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: schaffer_cos_variant(x)[1], "hess": lambda x: schaffer_cos_variant(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: schaffer_cos_variant(x)[1], "hess": lambda x: schaffer_cos_variant(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: schaffer_cos_variant(x)[1], "hess": lambda x: schaffer_cos_variant(x)[2]},
}

# Dicionário para armazenar tempos
method_times = {method: [] for method in methods}

# Loop de testes
for x0 in initial_points:
    print(f"\n==== Exercício 2 - Função Schaffer N.3 (x0 = {x0}) ====\n")

    results = {}
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: schaffer_cos_variant(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000  # em ms

        method_times[name].append(elapsed_time)
        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }

    # Exibição dos resultados
    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")
    print("\n" + "-" * 80 + "\n")

# Média dos tempos por método
print("\n==== Média de Tempo por Método ====\n")
print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
print("-" * 38)
for method, times in method_times.items():
    avg_time = sum(times) / len(times)
    print(f"{method:<20} | {avg_time:15.2f}")



==== Exercício 2 - Função Schaffer N.3 (x0 = [2.5  1.75]) ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   4.2480e-03 |    152 |    N/A |    N/A |       8.18 |     True
BFGS                 |   4.2480e-03 |     31 |     31 |    N/A |       6.06 |     True
Newton-CG            |   6.0113e-01 |      1 |      1 |      1 |       0.94 |     True
trust-ncg            |   4.2663e-03 |    401 |    368 |    368 |      28.09 |    False
trust-krylov         |   6.0113e-01 |      1 |      1 |      1 |       1.01 |    False
trust-exact          |   4.2663e-03 |    401 |    368 |    401 |      66.60 |    False

--------------------------------------------------------------------------------


==== Exercício 2 - Função Schaffer N.3 (x0 = [0. 0.]) ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------

## Exercício 3 - Função 142 - Streched V Senoidal Modificada

Avaliar para diferentes valores de D

$$
f(\mathbf{x}) = \sum_{i=1}^{D-1} 
\left( 0.5 x_i^2 + 1.5 x_{i+1}^2 \right)^{0.3} \cdot 
\left[ \sin^2 \left( 40 \cdot \left| x_i - x_{i+1} \right|^{0.2} \right) + 0.05 \right]
$$

In [8]:
import numpy as np
import time
from scipy.optimize import minimize

# Função Stretched V Sine Wave Modificada
def stretched_v_function(x):
    D = len(x)
    fval = 0.0
    for i in range(D - 1):
        xi = x[i]
        xi1 = x[i + 1]
        term_sum = np.clip(0.5 * xi ** 2 + 1.5 * xi1 ** 2, 1e-8, 1e3)
        diff = np.abs(xi - xi1)
        fval += term_sum ** 0.3 * (np.sin(40 * diff ** 0.2) ** 2 + 0.05)
    return fval

# Gradiente por diferenças finitas com segurança
def finite_diff_gradient(x):
    grad = np.zeros_like(x)
    eps = 1e-8
    for i in range(len(x)):
        x_forward = x.copy()
        x_backward = x.copy()
        x_forward[i] += eps
        x_backward[i] -= eps
        f_forward = stretched_v_function(x_forward)
        f_backward = stretched_v_function(x_backward)
        if not np.isfinite(f_forward) or not np.isfinite(f_backward):
            grad[i] = 0.0
        else:
            grad[i] = (f_forward - f_backward) / (2 * eps)
    return grad

# Métodos de otimização (sem hessiana)
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: finite_diff_gradient(x)},
}

method_times = {method: [] for method in methods}

# Executa as otimizações para D = 6 a 10
for D in range(6, 11):
    print(f"\n==== Stretched V Variant Results for D = {D} ====\n")
    x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
    if len(x0) < D:
        x0 = np.concatenate([x0, np.ones(D - len(x0))])
    else:
        x0 = x0[:D]

    results = {}
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: stretched_v_function(x), x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000
        method_times[name].append(elapsed_time)

        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "time": elapsed_time,
            "success": res.success
        }

    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")

# Média dos tempos
print("\n==== Média de Tempo por Método (D = 6 a 10) ====\n")
print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
print("-" * 38)
for method, times in method_times.items():
    avg_time = sum(times) / len(times)
    print(f"{method:<20} | {avg_time:15.2f}")



==== Stretched V Variant Results for D = 6 ====

Method               |        f(x*) |   nfev |   njev |  Time (ms) |  Success
-----------------------------------------------------------------------------
Nelder-Mead          |   3.3523e-01 |    243 |    N/A |      12.00 |     True
BFGS                 |   2.1750e-01 |     24 |     24 |      14.41 |     True

==== Stretched V Variant Results for D = 7 ====

Method               |        f(x*) |   nfev |   njev |  Time (ms) |  Success
-----------------------------------------------------------------------------
Nelder-Mead          |   1.5474e+00 |    335 |    N/A |      16.28 |     True
BFGS                 |   1.6302e-01 |     57 |     57 |      34.51 |     True

==== Stretched V Variant Results for D = 8 ====

Method               |        f(x*) |   nfev |   njev |  Time (ms) |  Success
-----------------------------------------------------------------------------
Nelder-Mead          |   4.6337e-01 |   1074 |    N/A |      60.52 |  

## Exercício 4 - Função 61 - Hansen

A partir de diferentes pontos iniciais, encontrar ao menos dois dos mínimos globais.

$$
f_{61}(x) \;=\; \left( \sum_{i=0}^{4} (i+1)\cos\left(ix_1 + i + 1\right) \right) \left( \sum_{j=0}^{4} (j+1)\cos\left((j+2)x_2 + j + 1\right) \right)
$$

$$
f_{61}^{\text{new}}(x_1, x_2) = \sin^2(x_1) + \cos^2(2x_2) + \frac{\sin(3x_1 x_2)}{1 + x_1^2 + x_2^2}
$$


In [11]:
import numpy as np
import time
from scipy.optimize import minimize

def hansen(x):
    x1, x2 = x[0], x[1]

    # Função
    sin_term = np.sin(x1)**2
    cos_term = np.cos(2 * x2)**2
    frac_term = np.sin(3 * x1 * x2) / (1 + x1**2 + x2**2)
    f = sin_term + cos_term + frac_term

    # Gradiente
    df_dx1 = 2 * np.sin(x1) * np.cos(x1) + \
             (3 * x2 * np.cos(3 * x1 * x2) * (1 + x1**2 + x2**2) - 2 * x1 * np.sin(3 * x1 * x2)) / (1 + x1**2 + x2**2)**2
             
    df_dx2 = -4 * np.sin(2 * x2) * np.cos(2 * x2) + \
             (3 * x1 * np.cos(3 * x1 * x2) * (1 + x1**2 + x2**2) - 2 * x2 * np.sin(3 * x1 * x2)) / (1 + x1**2 + x2**2)**2

    g = np.array([df_dx1, df_dx2])

    # Hessiana (diagonal apenas)
    d2f_dx1 = 2 * (np.cos(2 * x1)) + \
              ((-9 * x2**2 * np.sin(3 * x1 * x2)) * (1 + x1**2 + x2**2)**2 -
               2 * np.cos(3 * x1 * x2) * (1 + x1**2 + x2**2) * (3 * x2 + 2 * x1 * 3 * x1 * x2)) / (1 + x1**2 + x2**2)**3

    d2f_dx2 = -8 * np.cos(4 * x2) + \
              ((-9 * x1**2 * np.sin(3 * x1 * x2)) * (1 + x1**2 + x2**2)**2 -
               2 * np.cos(3 * x1 * x2) * (1 + x1**2 + x2**2) * (3 * x1 + 2 * x2 * 3 * x1 * x2)) / (1 + x1**2 + x2**2)**3

    # Hessiana (simplificada com aproximação cruzada)
    H = np.zeros((2, 2))
    H[0, 0] = d2f_dx1
    H[1, 1] = d2f_dx2
    H[0, 1] = H[1, 0] = df_dx1 * df_dx2 / f if f != 0 else 0

    return f, g, H

# Pontos iniciais
initial_points = [
    np.array([0.0, 0.0]),
    np.array([-2.0, 1.0]),
    np.array([3.0, -3.0]),
    np.array([1.5, 2.0]),
    np.array([-1.0, -2.0])
]

# Métodos de otimização
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: hansen_new(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: hansen_new(x)[1], "hess": lambda x: hansen_new(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: hansen_new(x)[1], "hess": lambda x: hansen_new(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: hansen_new(x)[1], "hess": lambda x: hansen_new(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: hansen_new(x)[1], "hess": lambda x: hansen_new(x)[2]},
}

method_times = {method: [] for method in methods}

# Execução para cada ponto inicial
for x0 in initial_points:
    print(f"\n==== Função Hansen - Mínimos Globais (x0 = {x0}) ====\n")
    
    results = {}
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: hansen_new(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000

        method_times[name].append(elapsed_time)
        
        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }

    # Exibe resultados
    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")
    print("\n" + "-" * 80 + "\n")

# Média dos tempos
print("\n==== Média de Tempo por Método ====\n")
print(f"{'Method':<20} | {'Avg Time (ms)':>15}")
print("-" * 38)
for method, times in method_times.items():
    avg_time = sum(times) / len(times)
    print(f"{method:<20} | {avg_time:15.2f}")



==== Função Hansen - Mínimos Globais (x0 = [0. 0.]) ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |  -3.0573e-01 |    117 |    N/A |    N/A |       6.42 |     True
BFGS                 |   1.0000e+00 |      1 |      1 |    N/A |       0.00 |     True
Newton-CG            |   1.0000e+00 |      1 |      1 |      1 |       0.00 |     True
trust-ncg            |   1.0000e+00 |      1 |      1 |      1 |       0.00 |     True
trust-krylov         |   1.0000e+00 |      1 |      1 |      1 |       1.00 |     True
trust-exact          |   1.0000e+00 |      1 |      1 |      1 |       0.00 |     True

--------------------------------------------------------------------------------


==== Função Hansen - Mínimos Globais (x0 = [-2.  1.]) ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------